In [1]:
import pandas as pd
from tqdm import tqdm
from copy import copy
import numpy as np
import json
import warnings
import xlrd
from operator import add
warnings.filterwarnings('ignore')

In [2]:
# Summation of the value insde a dictionary
def dictsum(myDict):      
    sum = 0
    for i in myDict: 
        sum = sum + myDict[i] 
    return sum

In [3]:
mapping = pd.read_excel('/mnt/c/Users/Shiyuan Wang/Desktop/Global_PM2.5/CEDS/CEDS_2017/EORA_CEDS_classification.xlsx',sheet_name='Sheet5',engine='openpyxl')

In [6]:
mapping

,EORA sectors,ISIC Rev.3 correspondence (EORA),Detailed working sectors (CEDS),2017 CEDS sectors,Unnamed: 4,Unnamed: 5
0,Agriculture,"01 – Agriculture, hunting and related service ...",Agriculture; 1A4c_Agriculture-forestry-fishing...,AGR/RCOO,NaN,NaN
1,Fishing,"Fishing, aquaculture and service activities in...",1A4c_Agriculture-forestry-fishing (c ),RCOO,NaN,NaN
2,Mining and Quarrying,10 – Mining of coal and lignite; extraction of...,1A2g_Ind-Comb-mining-quarrying (c ),IND,NaN,NaN
3,Food & Beverages,15 – Manufacture of food products and beverage...,1A2e_Ind-Comb-Food-tobacco (c ); 2H_Pulp-and-...,IND,NaN,NaN
4,Textiles and Wearing Apparel,17 – Manufacture of textiles;18 – Manufacture ...,1A2g_Ind-Comb-textile-leather (c),IND,NaN,NaN
5,Wood and Paper,20 – Manufacture of wood and of products of wo...,1A2g_Ind-Comb-wood-products (c ); 1A2d_Ind-Com...,IND,NaN,NaN
6,"Petroleum, Chemical and Non-Metallic Mineral P...","23 – Manufacture of coke, refined petroleum pr...",1A2c_Ind-Comb-Chemicals (c ); 2B_Chemical-indu...,IND,NaN,NaN
7,Metal Products,27 – Manufacture of basic metals;28 – Manufact...,1A2a_Ind-Comb-Iron-steel (c ); 1A2b_Ind-Comb-N...,IND,NaN,NaN
8,Electrical and Machinery,29 – Manufacture of machinery and equipment n....,1A2g_Ind-Comb-machinery (c ),IND,NaN,NaN
9,Transport Equipment,"34 – Manufacture of motor vehicles, trailers a...",1A2g_Ind-Comb-transpequip (c ),IND,NaN,NaN


In [7]:
sectors = list(mapping['EORA sectors'])
sectors

['Agriculture',
 'Fishing',
 'Mining and Quarrying',
 'Food & Beverages',
 'Textiles and Wearing Apparel',
 'Wood and Paper',
 'Petroleum, Chemical and Non-Metallic Mineral Products',
 'Metal Products',
 'Electrical and Machinery',
 'Transport Equipment',
 'Other Manufacturing',
 'Recycling',
 'Electricity, Gas and Water',
 'Construction',
 'Maintenance and Repair',
 'Wholesale Trade',
 'Retail Trade',
 'Hotels and Restraurants',
 'Transport',
 'Post and Telecommunications',
 'Finacial Intermediation and Business Activities',
 'Public Administration',
 'Education, Health and Other Services',
 'Private Households',
 'Others',
 'Re-export & Re-import']

In [ ]:
output_matrix = pd.read_csv('total_output_EORA_sector/EORA_output_matrix_2017.csv',index_col=0)
output_matrix = output_matrix.iloc[:4862,:187]
Countries = list(output_matrix.columns)
total_output_sector = list(output_matrix.sum(axis=1))

sectors = list(mapping['EORA sectors'])
# a dictionary of aggregated indices where each sector is aggregated across different countries

# Initialize an empty list to store the aggregated indices
aggregated_indices = {}

for sector_index in range(len(sectors)):
    sector = sectors[sector_index].strip()
    aggregated_indices[sector] = []
    for country_index in range(len(output_matrix.iloc[0])):
        # Calculate the index for the current sector in the current country
        index = sector_index + country_index * 26
        # Append the index to the list of aggregated indices
        aggregated_indices[sector].append(index)

# Total economic output from each country
total_output_country = [sum(total_output_sector[i:i+26]) for i in range(0, len(total_output_sector), 26)]

# The fraction of output from a sector of a country relative to the country's total output
total_output_fraction = {}
for sector_index in range(len(sectors)):
    output_list = [total_output_sector[i] for i in aggregated_indices[sectors[sector_index].strip()]]
    total_output_fraction[sectors[sector_index].strip()] = [a / b for a, b in zip(output_list, total_output_country)]

# check whether those sum up to be one
print(pd.DataFrame(total_output_fraction).sum(axis=1))

result_df = pd.DataFrame(total_output_fraction)

result_df.index = Countries
result_df.to_csv('total_output_fraction/total_output_fraction_2017.csv')